In [2]:
class EntryNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None
        self.prev = None


class DoubleLinkedList:
    def __init__(self):
        self.start = None
        self.end = None

    def append(self, key, value):
        new_entry = EntryNode(key, value)
        if not self.start:
            self.start = self.end = new_entry
        else:
            self.end.next = new_entry
            new_entry.prev = self.end
            self.end = new_entry

    def search(self, key):
        current = self.start
        while current:
            if current.key == key:
                return current
            current = current.next
        return None

    def delete(self, key):
        current = self.start
        while current:
            if current.key == key:
                if current.prev:
                    current.prev.next = current.next
                else:
                    self.start = current.next
                if current.next:
                    current.next.prev = current.prev
                else:
                    self.end = current.prev
                return True
            current = current.next
        return False

    def __len__(self):
        count = 0
        current = self.start
        while current:
            count += 1
            current = current.next
        return count


class HashMap:
    def __init__(self, initial_size=8, hash_func=None):
        self.size = initial_size
        self.buckets = [DoubleLinkedList() for _ in range(self.size)]
        self.item_count = 0
        self.expand_limit = 0.75  # Expand when over 75% full
        self.shrink_limit = 0.25  # Shrink when under 25% full
        self.hash_function = hash_func if hash_func else self.basic_hash_function

    def basic_hash_function(self, key):
        A = (5 ** 0.5 - 1) / 2  # Multiplier for the multiplication method
        return int((self.size * ((key * A) % 1))) % self.size

    def resize(self, new_size):
        old_buckets = self.buckets
        self.size = new_size
        self.buckets = [DoubleLinkedList() for _ in range(self.size)]
        self.item_count = 0
        for bucket in old_buckets:
            current = bucket.start
            while current:
                self.insert(current.key, current.value)
                current = current.next

    def insert(self, key, value):
        bucket_index = self.hash_function(key)
        bucket = self.buckets[bucket_index]
        entry = bucket.search(key)
        if entry:
            entry.value = value  # Update existing value
        else:
            bucket.append(key, value)
            self.item_count += 1

        # Resize if needed
        if self.item_count / self.size > self.expand_limit:
            self.resize(self.size * 2)

    def remove(self, key):
        bucket_index = self.hash_function(key)
        bucket = self.buckets[bucket_index]
        if bucket.delete(key):
            self.item_count -= 1

        # Resize if needed
        if self.item_count / self.size < self.shrink_limit and self.size > 8:
            self.resize(self.size // 2)

    def retrieve(self, key):
        bucket_index = self.hash_function(key)
        bucket = self.buckets[bucket_index]
        entry = bucket.search(key)
        if entry:
            return entry.value
        return None

    def __len__(self):
        return self.item_count

    def show(self):
        for idx, bucket in enumerate(self.buckets):
            current = bucket.start
            print(f"Bucket {idx}:", end=" ")
            while current:
                print(f"({current.key}: {current.value})", end=" -> ")
                current = current.next
            print("None")


# Example Usage
hash_map = HashMap()

# Insert key-value pairs
hash_map.insert(10, 100)
hash_map.insert(20, 200)
hash_map.insert(30, 300)
hash_map.insert(40, 400)

# Display the hash map
hash_map.show()

# Retrieve a value
print("Value for key 30:", hash_map.retrieve(30))

# Remove a key
hash_map.remove(20)

# Display the hash map after removal
hash_map.show()


Bucket 0: None
Bucket 1: (10: 100) -> None
Bucket 2: (20: 200) -> None
Bucket 3: None
Bucket 4: (30: 300) -> None
Bucket 5: (40: 400) -> None
Bucket 6: None
Bucket 7: None
Value for key 30: 300
Bucket 0: None
Bucket 1: (10: 100) -> None
Bucket 2: None
Bucket 3: None
Bucket 4: (30: 300) -> None
Bucket 5: (40: 400) -> None
Bucket 6: None
Bucket 7: None
